# 百度网盘AI大赛-模糊文档图像恢复比赛第1名方案

非常幸运获得了百度网盘大赛，[模糊文档图像恢复比赛](https://aistudio.baidu.com/aistudio/competition/detail/349/0/leaderboard)AB榜都第一的好成绩！

特别感谢，txyugood提供的[Restormer_paddle](https://github.com/txyugood/Restormer_Paddle)的代码，有效的帮助我们构建了baseline和调参调优。

# 一、赛题分析
此次大赛主题结合日常生活常见情景展开，当使用移动设备扫描获取文档的过程中，很多是文字、字母和数字等符号内容。通过拍摄截取等方式获取文档，就非常有可能导致内容模糊、噪音叠加等问题的发生，使得无法实际发挥作用。期望同学们通过计算机技术，帮助人们将模糊失焦的文档恢复清晰，提高使用便捷度和效率。



# 二、 数据分析
- 本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共1000对样本，A榜测试集共200个样本，B榜测试集共200个样本；抽取一部分数据如图：
![](https://ai-studio-static-online.cdn.bcebos.com/347f2446875d40d9968da672f483976b7f1e78c7dc594ffab21a45f7c82e792c)
![](https://ai-studio-static-online.cdn.bcebos.com/978ef62d463343f6b4a5e95d81f34cbcc5050a42ee8f460484bdc7de0d5b3937)
![](https://ai-studio-static-online.cdn.bcebos.com/1e1986723a4b413d973722ccbea4a4d3606c22f05d8d4e9fad2582a123d6f39e)

- blur_image 为模糊的文档图像数据，gt_image 为非模糊的真值数据（仅有训练集数据提供gt_image ，A榜测试集、B榜测试集数据均不提供gt_image）；
- blur_image 与 gt_image 中的图片根据图片名称一一对应。
- 进一步分析训练数据，计算训练数据的平均大小和测试数据的平均大小，我们会发现训练数据的图像大小为测试数据大小的大约2倍左右。因此训练数据和测试数据不在一个domain，因此训练时我们针对训练图像进行了下采样两倍的处理。

# 二、评价标准
评价指标为 PSNR 和 MSSSIM；

用于评价的机器环境仅提供两种框架模型运行环境：paddlepaddle 和 onnxruntime，其他框架模型可转换为
上述两种框架的模型；

机器配置：V100，显存16G，内存10G；

单张图片耗时>1.2s，决赛中的性能分数记0分。

由评价标准可知，不能使用大模型。

# 三、算法介绍

Baseline的选择：针对图像去模糊这个任务，我们首先查询了paper with code网站，找到了目前优异的几个模型：MADANet、NAFNet以及Restormer。根据paper with code的信息我们可以了解到MADANet采用了额外数据进行训练，并且这个赛道数据量并不是很多。因此我们采用了NAFNet作为我们此次的baseline。
网络主体架构为UNet，如图：

![](https://ai-studio-static-online.cdn.bcebos.com/c2ced773bf7b4d9db72ed48c4b92999964dd4198103c463282fa46a62cd9d319)

其中Encoder和Decoder采用NAFBlock:

![](https://ai-studio-static-online.cdn.bcebos.com/a8b2262aaa144f8ea8c9c69087b0f93d988fc976e70e4e8ca61b2c0e88f274df)


由于比赛推理速度需要每张图像1.2s之内，我们对原有的NAFNet进行了通道缩减和减少深度的操作来使得A榜测试图像推理能达到0.8305秒每张图，图中各个进行下采样的block数量设置为1，最深层的block数量设置为10，在单卡V100上训练600000次迭代。为了进一步提升网络性能，我们还进一步采用了Test-time Local Converter (TLC)。主要参考了[Improving Image Restoration by Revisiting Global Information Aggregation论文](https://arxiv.org/abs/2112.04491)




# 四、数据增强与清洗

### 数据划分
官方给的数据为1000张图像，我们将最后的100张图像作为测试集，前面900张作为训练集

### 数据增广
- 我们对训练集进行了裁剪为1024大小的patch。
- 在训练模型的过程中，为了进一步充分利用所有的数据，我们采用了图像翻转，图像旋转等操作来增广数据集。
- 为了加快训练过程，我们还采用了随机裁剪策略。

### 数据清洗
我们发现在裁剪过程中，会裁剪出很多空白图块以及一些信息很少的图块。这些图块并不会帮助网络学习到对应去模糊的知识，因此我们针对这样的情况进行了数据清洗。由于裁剪后的数据有7w+的数据量，筛选非常困难，并且在Aistudio上进行训练的话，需要不断重新导入数据。因此，我们考虑到计算每个图块的平均梯度来度量每个图块包含复杂纹理的多少，并且进行统计。如下所示：

![](https://ai-studio-static-online.cdn.bcebos.com/d7f532be7a444bbfafd4b19c50787c2865b97e7032604963bc511060aaea38b6)


从统计图中我们不难看出，大多数图块都分布在20左右，低于10的图块较少。我们在进一步比对裁剪后的数据，发现图像平均梯度小于10的图块大多都是空白以及一些纹理少的图块。因此，我们直接将平均梯度大于10的图块地址写入txt文件中，通过读取txt文件中的地址来读取数据，然后构建训练集。

计算梯度主要代码如下：
```
import cv2
import numpy as np
def cal_gradient(img_path):
    img = cv2.imread(img_path)
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
    sobelx = cv2.convertScaleAbs(sobelx)
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
    sobely = cv2.convertScaleAbs(sobely)
    sobelxy = cv2.addWeighted(sobelx,0.5,sobely,0.5,0)
    return np.mean(sobelxy)
```


# 五、训练细节
### 训练配置
- 总迭代数：600000 iteration
- 我们采用了渐进训练的方式，来加速训练过程。分别采用batch size为8和patch size为192来进行训练184000次迭代，batch size为5和patch size为256来进行训练128000次迭代，batch size为4和patch size为320来进行训练96000次迭代，batch size为2和patch size为384来进行训练72000次迭代，batch size为1和patch size为512来进行训练72000次迭代，batch size为1和patch size为1024来进行训练48000次迭代。
- 我们采用了余弦退火的学习率策略来优化网络，在184000次迭代以及416000迭代处进行初始学习率
- 学习率我们采用2e-4，优化器为AdamW

### 训练分为阶段：
- 第一阶段损失函数为L1Loss
- 第二阶段损失函数使用Charbonnier Loss+MS_SSIMLoss
- 最后，在全量数据进行finetuning

主要Loss函数如下：
```
def _ssim(img1, img2, window, window_size, channel=3 ,data_range = 255.,size_average=True,C=None):
    # size_average for different channel

    padding = window_size // 2

    mu1 = F.conv2d(img1, window, padding=padding, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padding, groups=channel)
    # print(mu1.shape)
    # print(mu1[0,0])
    # print(mu1.mean())
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=padding, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padding, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padding, groups=channel) - mu1_mu2
    if C ==None:
        C1 = (0.01*data_range) ** 2
        C2 = (0.03*data_range) ** 2
    else:
        C1 = (C[0]*data_range) ** 2
        C2 = (C[1]*data_range) ** 2
    # l = (2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1)
    # ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    sc = (2 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    lsc = ((2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1))*sc

    if size_average:
        ### ssim_map.mean()是对这个tensor里面的所有的数值求平均
        return lsc.mean()
    else:
        # ## 返回各个channel的值
        return lsc.flatten(2).mean(-1),sc.flatten(2).mean(-1)

def ms_ssim(
    img1, img2,window, data_range=255, size_average=True, window_size=11, channel=3, sigma=1.5, weights=None, C=(0.01, 0.03)
):

    r""" interface of ms-ssim
    Args:
        img1 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        img2 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        data_range (float or int, optional): value range of input images. (usually 1.0 or 255)
        size_average (bool, optional): if size_average=True, ssim of all images will be averaged as a scalar
        win_size: (int, optional): the size of gauss kernel
        win_sigma: (float, optional): sigma of normal distribution
        win (torch.Tensor, optional): 1-D gauss kernel. if None, a new kernel will be created according to win_size and win_sigma
        weights (list, optional): weights for different levels
        K (list or tuple, optional): scalar constants (K1, K2). Try a larger K2 constant (e.g. 0.4) if you get a negative or NaN results.
    Returns:
        torch.Tensor: ms-ssim results
    """
    if not img1.shape == img2.shape:
        raise ValueError("Input images should have the same dimensions.")

    # for d in range(len(img1.shape) - 1, 1, -1):
    #     img1 = img1.squeeze(dim=d)
    #     img2 = img2.squeeze(dim=d)

    if not img1.dtype == img2.dtype:
        raise ValueError("Input images should have the same dtype.")

    if len(img1.shape) == 4:
        avg_pool = F.avg_pool2d
    elif len(img1.shape) == 5:
        avg_pool = F.avg_pool3d
    else:
        raise ValueError(f"Input images should be 4-d or 5-d tensors, but got {img1.shape}")

    smaller_side = min(img1.shape[-2:])

    assert smaller_side > (window_size - 1) * (2 ** 4), "Image size should be larger than %d due to the 4 downsamplings " \
                                                        "with window_size %d in ms-ssim" % ((window_size - 1) * (2 ** 4),window_size)

    if weights is None:
        weights = [0.0448, 0.2856, 0.3001, 0.2363, 0.1333]
    weights = paddle.to_tensor(weights)

    if window is None:
        window = create_window(window_size, sigma, channel)
    assert window.shape == [channel, 1, window_size, window_size], " window.shape error"

    levels = weights.shape[0] # 5
    mcs = []
    for i in range(levels):
        ssim_per_channel, cs =  _ssim(img1, img2, window=window, window_size=window_size,
                                       channel=3, data_range=data_range,C=C, size_average=False)
        if i < levels - 1:
            mcs.append(F.relu(cs))
            padding = [s % 2 for s in img1.shape[2:]]
            img1 = avg_pool(img1, kernel_size=2, padding=padding)
            img2 = avg_pool(img2, kernel_size=2, padding=padding)

    ssim_per_channel = F.relu(ssim_per_channel)  # (batch, channel)
    mcs_and_ssim = paddle.stack(mcs + [ssim_per_channel], axis=0)  # (level, batch, channel) 按照等级堆叠
    ms_ssim_val = paddle.prod(mcs_and_ssim ** weights.reshape([-1, 1, 1]), axis=0) # level 相乘
    #print(ms_ssim_val.shape)
    if size_average:
        return ms_ssim_val.mean()
    else:
        # 返回各个channel的值
        return ms_ssim_val.flatten(2).mean(1)
class MS_SSIMLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self, loss_weight, reduction='mean', data_range=255., channel=3, window_size=11, sigma=1.5):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(MS_SSIMLoss, self).__init__()
       self.data_range = data_range
       self.C = [0.01, 0.03]
       self.window_size = window_size
       self.channel = channel
       self.sigma = sigma
       self.window = create_window(self.window_size, self.sigma, self.channel)
       self.loss_weight = loss_weight
       # print(self.window_size,self.window.shape)
   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用Paddle中相关API自定义的计算逻辑
       # output = xxxxx
       # return output
       return self.loss_weight * (1-ms_ssim(input, label, data_range=self.data_range,
                      window = self.window, window_size=self.window_size, channel=self.channel,
                      size_average=True,  sigma=self.sigma,
                      weights=None, C=self.C))
class CharbonnierLoss(nn.Layer):
    """Charbonnier Loss (L1)"""

    def __init__(self, loss_weight=1.0, reduction='mean', eps=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps

    def forward(self, x, y):
        diff = x - y
        # loss = torch.sum(torch.sqrt(diff * diff + self.eps))
        loss = paddle.mean(paddle.sqrt((diff * diff) + (self.eps*self.eps)))
        return loss

class CompositeLoss(nn.Layer):
    """Charbonnier Loss (L1)"""

    def __init__(self, loss_weight=1.0, reduction='mean', eps=1e-3):
        super(CompositeLoss, self).__init__()
        self.SSIMLoss = MS_SSIMLoss(loss_weight=1.0)
        self.L2Loss = CharbonnierLoss()

    def forward(self, x, y):
        loss = self.SSIMLoss(x, y) + self.L2Loss(x, y)
        return loss
```

# 六、测试细节
由于测试图像大多为2K图像，直接预测图像会导致显存不足的问题，因此我们采用了裁剪预测的方式来进行，将每张输入图像在输入网络前平均裁剪成大小一致的4块，不能被2整除的图像进行padding再裁剪，最后通过网络预测出四个图像块，然后再通过拼接操作，拼接成输出图像来输出。

# 七、代码结构
### code:
- config: 配置文件
- data: 定义数据增强函数
- models: 定义网络模型以及损失函数
- metrics: 定义评价指标函数
- utils: 定义其他函数
- dataset.py: 定义数据集
- output: 模型训练输出文件夹
- data_preparation.py: 训练数据裁剪脚本
- data_preparation_all.py: 全量训练数据裁剪脚本
- generate_meta_info.py: 数据筛选脚本
- generate_meta_info_all.py: 全量数据筛选脚本
- predict.py: 预测脚本
- train.py: 训练脚本
### test_code:
- predict.py: 预测脚本
- model.pdparams: 最佳权重

# 八、上分策略

![](https://ai-studio-static-online.cdn.bcebos.com/2bffa4513a1f498da0945379160fa87e420dd753765d4604bc5071e76e07de34)


上分策略主要集中在数据清洗、推理速度优化和损失函数优化:

### 数据清洗
可以看出测试数据和训练数据的domain不一致，因此采用下采样两倍后的训练数据可以极大提升网络性能。同时不必要的空白图块也会干扰网络学习到去模糊的知识，让网络误以为恢复空白图块可以恢复的很好导致收敛不充分。因此采用了数据筛选后，我们的模型性能可以进一步提升。

### 推理速度

原始代码推理速度1.67s,耗时严重，经分析，我们采用更小的模型，在精度损失不大的情况下可以获得0.83s的速度， 网络满足线上V100推理显存需求

### 损失函数优化
一个很明显的情况是线上PSNR的指标很高，因此需要考虑如何提升ms_ssim指标。在网络第一阶段收敛后，结合Charbonnier Loss和MS_SSIM Loss进行分数性能的提升。

### Test-time Local Converter (TLC)优化
由于训练是在一个固定的patch size上进行训练的，这导致在处理大图块的过程中，不能充分利用所有信息，因此采用了TLC的模型优化策略来进一步提升网络模型。同时也导致了一定的时间损耗。

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 代码启动过程
## 训练过程

In [1]:
### 解压数据
!cd data/data154440/ && unzip -q deblur_test.zip
!cd data/data154440/deblur_test/ && rm -r -f deblur_testA.zip
!cd data/data154440/deblur_test/ && unzip -q '*.zip'
!cd data/data154440/deblur_test/ && rm -r -f __MACOSX


10 archives were successfully processed.


In [2]:
## 去除不必要的zip文件
! cd data/data154440/deblur_test/ && rm -r -f *.zip

In [3]:
## 创建训练数据目录
! mkdir data/data154440/Train_data/
! mkdir data/data154440/Train_data/gt/
! mkdir data/data154440/Train_data/input/
! mkdir data/data154440/Val_data

In [4]:
## 数据整理
! mv data/data154440/deblur_test/*/gt_image/* data/data154440/Train_data/gt/
! mv data/data154440/deblur_test/*/blur_image/* data/data154440/Train_data/input/

In [ ]:
## 数据预处理
! cd work/code/ && python data_preparation.py
! cd work/code/ && python data_preparation_all.py

In [ ]:
## 数据筛选
! cd work/code/ && python generate_meta_info.py
! cd work/code/ && python generate_meta_info_all.py

In [ ]:
## 安装训练所需的python包
! pip install scikit-image

In [ ]:
## 利用L1Loss训练第一阶段
! cd work/code/ && python train.py -opt configs/NAFNet_phase1.yml

In [ ]:
## 利用Charbonnier Loss+MS_SSIMLoss进行训练第二阶段
! cd work/code/ && python train.py -opt configs/NAFNet_phase2.yml --resume output/model/last_model

In [ ]:
## 利用Charbonnier Loss+MS_SSIMLoss对全量数据进行finetuning
! cd work/code/ && python train.py -opt configs/NAFNet_fintune_all.yml --resume output/model/best_model

## 预测过程
按照官方给定的预测脚本运行方式相同，将最终的finetuning模型改名为model.pdparams，并且放在predict.py的同意目录下，使用命令：
```
python predict.py [src_image_dir] [results]
```

In [ ]:
## 测试脚本
! cd work/code/ && python predict.py {your_test_data_path} {save_path}